**Anna Smith**

**GitHub username: acs-233**

**Imperial College London - MSc EDSML - IRP**
# **Node Map Explore**

In [2]:
from pathlib import Path
import pandas as pd
from bayareaco2.preprocessing.features import plot_nodes_gdf

In [2]:
path = Path.cwd().parent.parent.parent / 'Data' / 'CO2' / 'node_locations.csv'
node_locations = pd.read_csv(path)

In [3]:
nodes_map = plot_nodes_gdf(node_locations)

In [4]:
nodes_map

In [5]:
path = Path.cwd().parent.parent / 'explore' / 'nodes.html'
nodes_map.save(path)
print(f'Saved map to {path}')

Saved map to /Users/annasmith/Desktop/EDSML/IRP/Code/bayareaco2/notebooks/explore/nodes.html


In [5]:
path = (
    Path.cwd().resolve().parent.parent.parent
    / "Data"
    / "CO2"
    / "daily_avg_BEACO2N.csv"
)
daily_avg = pd.read_csv(path)

In [17]:
daily_avg['date'].value_counts()

date
2023-03-04    41
2023-03-03    41
2022-07-29    40
2022-07-28    40
2022-08-02    40
              ..
2024-04-17     1
2024-04-18     1
2024-04-19     1
2024-04-20     1
2024-05-27     1
Name: count, Length: 2842, dtype: int64

In [20]:
top_daily_avg = daily_avg[daily_avg['date'] == '2023-03-04']

In [46]:
top_daily_avg['co2'].describe()

count     41.000000
mean     427.450334
std        3.935696
min      420.825597
25%      425.493042
50%      426.476550
75%      428.366353
max      444.156851
Name: co2, dtype: float64

In [61]:
avg_co2 = daily_avg.groupby('node_id')[['co2', 'lat', 'lng']].mean().reset_index()

In [62]:
avg_co2

,node_id,co2,lat,lng
0,2,418.549088,38.289,-122.503
1,3,436.968947,37.824,-122.233
2,4,433.903186,37.831,-122.213
3,5,425.356544,37.796,-122.279
4,7,413.858064,37.798,-122.161
...,...,...,...,...
60,96,407.884634,37.617,-122.069
61,125,425.619893,37.772,-122.392
62,280,431.680587,37.873,-122.268
63,281,430.650548,37.876,-122.267


In [76]:
import folium
import branca.colormap as cm

map_center = [37.955, -122.2]
nodes_map = folium.Map(location=map_center, zoom_start=10)

linear = cm.LinearColormap(
    colors=["green", "yellow", "red"],
    index=[410, 425, 440],
    vmin=410,
    vmax=440,
)

for _, row in avg_co2.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lng"]],
        radius=6,
        color="black",
        fill=True,
        fill_color=linear(row["co2"]),
        fill_opacity=0.8,
        popup=folium.Popup(
            f"""
            <div style="width: 82px; padding: 5px; font-size: 12px;">
                <b>node_id: {int(row['node_id'])}</b><br>
                co2: {row['co2']:.2f}<br>
            </div>
            """,
            max_width=300,
        ),
    ).add_to(nodes_map)

# Creating colorbar for CO2 values
colorbar_html = linear._repr_html_().replace(
    "height: 20px;", "height: 40px; width: 80px; margin: 5px auto;"
)

# Creating legend for node categories and colorbar for R² values
legend_html = f"""
    <div style="position: fixed; 
    bottom: 0px; left: 100px; width: 475px; height: auto; 
    border:2px solid grey; background-color:white;
    z-index:9999; font-size:14px; padding:10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.5);
    ">
    &nbsp; <b>Average CO2 concentration (ppm)</b><br>
    {colorbar_html}
    </div>
"""

# Add legend to the map
folium.Marker(
    location=[avg_co2["lat"].mean(), avg_co2["lng"].mean()],
    icon=folium.DivIcon(html=legend_html),
).add_to(nodes_map)

output_html_path = (
            Path.cwd().parent.parent / 'explore' / 'co2.html'
        )

nodes_map.save(output_html_path)

nodes_map